In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
# Create sqlalchemy
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///spotify.sqlite")

In [3]:
# Connect to database
with engine.connect() as conn:
    songs_df = pd.read_sql("SELECT * FROM songs", conn)

In [4]:
songs_df.head()

,track_id,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,hit,genre
0,4pR4oQJULf7FDB54TleMyH,0,0.443,0.778,-7.564,0.2660,0.241,0.000000,0.2150,0.628,128.250,1.0,Hip_Hop_R&B
1,4Ds4bq6aqOSAp1T7DikOi4,0,0.580,0.800,-7.528,0.3450,0.281,0.000000,0.0708,0.483,81.303,0.0,Hip_Hop_R&B
2,7nYbKsvhM88mHCmFsAIQVp,0,0.805,0.746,-5.211,0.1850,0.238,0.000000,0.2150,0.717,127.922,0.0,Hip_Hop_R&B
3,7gVNP7rI9UBZndge0ulKfL,0,0.582,0.660,-4.988,0.0544,0.399,0.000000,0.1270,0.484,83.789,0.0,Hip_Hop_R&B
4,1hLvWelTny8vttEEZIXVjw,0,0.776,0.314,-9.513,0.0340,0.881,0.000723,0.1390,0.476,124.055,0.0,Hip_Hop_R&B


In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(songs_df.genre.values.reshape(-1,1)))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(['genre'])
encode_df.head()

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,genre_Cinema,genre_Classical,genre_Country,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Merge the dataframes and drop genre
songs_df = songs_df.merge(encode_df, left_index=True, right_index=True).drop("genre",1)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [7]:
songs_df.head()

,track_id,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,4pR4oQJULf7FDB54TleMyH,0,0.443,0.778,-7.564,0.2660,0.241,0.000000,0.2150,0.628,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4Ds4bq6aqOSAp1T7DikOi4,0,0.580,0.800,-7.528,0.3450,0.281,0.000000,0.0708,0.483,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7nYbKsvhM88mHCmFsAIQVp,0,0.805,0.746,-5.211,0.1850,0.238,0.000000,0.2150,0.717,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7gVNP7rI9UBZndge0ulKfL,0,0.582,0.660,-4.988,0.0544,0.399,0.000000,0.1270,0.484,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1hLvWelTny8vttEEZIXVjw,0,0.776,0.314,-9.513,0.0340,0.881,0.000723,0.1390,0.476,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Scale tempo and loudness

scaled_features = songs_df.copy()
col_names = ['loudness', 'tempo']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

scaled_features[col_names] = features
scaled_features.head()

,track_id,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,4pR4oQJULf7FDB54TleMyH,0,0.443,0.778,0.094609,0.2660,0.241,0.000000,0.2150,0.628,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4Ds4bq6aqOSAp1T7DikOi4,0,0.580,0.800,0.102129,0.3450,0.281,0.000000,0.0708,0.483,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7nYbKsvhM88mHCmFsAIQVp,0,0.805,0.746,0.586068,0.1850,0.238,0.000000,0.2150,0.717,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7gVNP7rI9UBZndge0ulKfL,0,0.582,0.660,0.632645,0.0544,0.399,0.000000,0.1270,0.484,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1hLvWelTny8vttEEZIXVjw,0,0.776,0.314,-0.312468,0.0340,0.881,0.000723,0.1390,0.476,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = scaled_features['hit'].values
X = scaled_features.drop(['hit', 'track_id'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 54
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 54)                1296      
                                                                 
 dense_1 (Dense)             (None, 30)                1650      
                                                                 
 dense_2 (Dense)             (None, 15)                465       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 3,427
Trainable params: 3,427
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [14]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5495 - accuracy: 0.7389
Epoch 2/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5403 - accuracy: 0.7404
Epoch 3/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5388 - accuracy: 0.7402
Epoch 4/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5375 - accuracy: 0.7404
Epoch 5/100
2181/2204 [============================>.] - ETA: 0s - loss: 0.5372 - accuracy: 0.7407
Epoch 5: saving model to checkpoints\weights.05.hdf5
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5370 - accuracy: 0.7408
Epoch 6/100
2204/2204 [==============================] - 3s 2ms/step - loss: 0.5357 - accuracy: 0.7404
Epoch 7/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5344 - accuracy: 0.7410
Epoch 8/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5327 - accuracy: 0.7416
Epoch 9/100
2204/2204 [=============

2204/2204 [==============================] - 3s 1ms/step - loss: 0.5079 - accuracy: 0.7540
Epoch 64/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5074 - accuracy: 0.7536
Epoch 65/100
2183/2204 [============================>.] - ETA: 0s - loss: 0.5073 - accuracy: 0.7540
Epoch 65: saving model to checkpoints\weights.65.hdf5
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5074 - accuracy: 0.7540
Epoch 66/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5071 - accuracy: 0.7545
Epoch 67/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5070 - accuracy: 0.7543
Epoch 68/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5064 - accuracy: 0.7547
Epoch 69/100
2204/2204 [==============================] - 3s 1ms/step - loss: 0.5061 - accuracy: 0.7540
Epoch 70/100
2198/2204 [============================>.] - ETA: 0s - loss: 0.5058 - accuracy: 0.7546
Epoch 70: saving model to checkpoints\weight

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

735/735 - 1s - loss: 0.5089 - accuracy: 0.7517 - 773ms/epoch - 1ms/step
Loss: 0.508899986743927, Accuracy: 0.7517446875572205


In [16]:
# Export model to HDF5 file
nn.save("Spotify_ml.h5")